<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/suh/LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#access huggingface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
class Chat:

  def __init__(self, prompt) :
    self.prompt= prompt
    #load model
    self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
    self.model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
    #create a transformer pipeline for text generation
    self.pipeline = transformers.pipeline(
    "text-generation",

    model=self.model,

   tokenizer=self.tokenizer,
  torch_dtype=torch.float16,

    device_map="auto",

  )


#given a prompt it will add the specific instructions that will give the output in the correct format
#the current additions use few-shot prompt
  def add_template(self):

    #this is the main instruction of what is being done
    instruction= 'The user is trying to find a word that matches the description in the prompt. The answer is 5 possible words that the prompt is trying to describe. The following are examples of Prompt-Answer format.\n'  # eg Given the context provided, answer the question

    #this is the few shot examples being passed
    few_shot = "Prompt: it’s a type of bird, kind of rare. They’re very colorful, with long tails, and they can mimic sounds. They live in tropical areas\n Answer:  Hummingbird, Parrot, Finch, Toucan, Macaw\n Prompt: It’s a type of bird, very colorful, and it can mimic human speech. People sometimes keep them as pets. They live in tropical areas.\nAnswer: Parrot, Macaw, Cockatoo, Canary, Finch\nPrompt: I need the...you know...the thing for writing\nAnswer: Pen, Pencil, Quill, Fountain Pen, Crayon\nPrompt : I want to go to the...um...place where we get groceries\n  Answer: Store, Supermarket, Market, Grocery, Shop\nPrompt : "

    #this is the specific instruction given to the model to generate the words
    system_prompt = 'Now it is your turn to give the answer to the last prompt. Give 30 possible words being described instead of just 5 like in the examples. It is important that you give 30 words. It is very important that you only provide the final output without any additional comments or remarks or notes.'

    chat_prompt= instruction + few_shot + self.prompt + system_prompt
    return chat_prompt

  def clean_generated(self, gen):
    #first strip the generated answer from 'Answer:'
    gen.replace('Answer: ', '')
    #next split the text by \n
    x = gen.split('\n')
    #it is possible the current line is in the format : word1, word2, word3, ...-> split on ,
    possible_words = x[0].split(',')

    return possible_words

#this make will take the refined prompt and pass it to the LLM
  def get_suggestions(self, chat_prompt):
    sequences = self.pipeline(
    self.add_template(),

    do_sample=True,

    top_k=20,

    num_return_sequences=1,

    eos_token_id=self.tokenizer.eos_token_id,

    max_length=800,

    temperature= 0.3,

    return_full_text= False

    )
    sugg= self.clean_generated(sequences['generated_text'])

    return sugg



In [ ]:
### wrapper function
#creates an instance of chat
#runs the llm tp generate an answer
# cleans the generated answer into a list and return the list

import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_list(prompt):
  chat = Chat(prompt)
  return chat.get_suggestions()
